一组预测器称为集成, 这种技术也被称为集成学习, 而一个集成学习算法则被称为集成方法。

# 7.1 投票分类器

已经训练好了一些分类器，要创建出一个更好的分类器，最简单的方法就是聚合每个分类器的预测，然后将得票最多的结果作为预测类别  
这种大多数投票分类器被称为硬投票分类器

创建并训练一个投票分类器，由三种不同的分类器组成，训练集是卫星数据集

In [3]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=10000, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)

from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.876
RandomForestClassifier 0.9876666666666667
SVC 0.989
VotingClassifier 0.986


# 7.2 bagging和pasting

bagging: 每个预测器使用的算法相同，但在不同的训练集随机子集上进行训练  
pasting: 采样时样本不放回，bagging允许训练实例被同一个预测器多次采样

与直接在原始训练集上训练的单个预测器相比，集成的偏差相近，但是方差更低

## 7.2.1 Scikit-Learn中的bagging和pasting

In [7]:
# 训练一个包含500个决策树分类器的集成
# 每次从训练集随机采样100个训练实例，然后放回
# 如果使用pasting，只需设置bootstrap=False
# n_jobs指示多少CPU内核进行训练和预测(-1表示所有可用)

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), 
    n_estimators=500,
    max_samples=100, 
    bootstrap=True, 
    # n_jobs=-1         !会报错
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

## 7.2.2 包外评估

未被采样的训练实例称为包外(oob)实例  
设置oob_score=True在训练结束自动进行包外评估

In [9]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), 
    n_estimators=500,
    max_samples=100, 
    bootstrap=True, 
    # n_jobs=-1         !会报错
    oob_score=True
)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9735714285714285

In [10]:
# 验证
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9686666666666667

测试集上获得96.86%的准确率，与oob_score接近

In [11]:
# 每个训练实例的包外决策函数也可以通过变量oob_decision_function_获得

bag_clf.oob_decision_function_

array([[0.93209877, 0.06790123],
       [0.        , 1.        ],
       [0.99389002, 0.00610998],
       ...,
       [0.00608519, 0.99391481],
       [0.08793456, 0.91206544],
       [0.98785425, 0.01214575]])

# 7.3 随机补丁和随机子空间

BaggingClassifier类也支持对特征进行采样  
采样由两个超参数控制:max_features和bootstrap_features  
对高维输入(例如图像)特别有用